In [2]:
import os, sqlite3, urllib.parse
import pandas as pd
from datetime import datetime, timezone

# ===== CONFIG =====
DB_PATH = r"C:\Users\chris\boxingproject\boxing_odds_staging.db"
OUT_DIR = r"C:\Users\chris\boxingproject"
UPCOMING_FILE = os.path.join(OUT_DIR, "upcoming_fights.html")
PROFILE_FILE  = os.path.join(OUT_DIR, "fighter_profile.html")
POSSIBLE_DATE = "2025-12-31"  # ignore these entirely

# ===== LOAD LATEST SNAPSHOT =====
if not os.path.exists(DB_PATH):
    raise SystemExit(f"❌ DB not found: {DB_PATH}")

conn = sqlite3.connect(DB_PATH)
latest_sql = """
SELECT *
FROM odds_history
WHERE (event_id, fighter, bookmaker, observed_at) IN (
    SELECT event_id, fighter, bookmaker, MAX(observed_at)
    FROM odds_history
    GROUP BY event_id, fighter, bookmaker
)
"""
df = pd.read_sql(latest_sql, conn)
conn.close()

if df.empty:
    raise SystemExit("❌ No odds in odds_history. Run your ingest first.")

# ===== CLEAN / PREP =====
df["commence_time"] = pd.to_datetime(df["commence_time"], errors="coerce", utc=True)
df["Fight Date"] = df["commence_time"].dt.strftime("%Y-%m-%d")
df["Fight Time (UTC)"] = df["commence_time"].dt.strftime("%H:%M")
df["Matchup"] = df["home_team"].fillna("") + " vs " + df["away_team"].fillna("")
df["fighter"] = df["fighter"].astype(str).str.strip()

# H2H only and no draws
df = df[(df["market"].astype(str).str.lower() == "h2h") & (df["fighter"].str.lower() != "draw")].copy()
# Remove placeholders
df = df[df["Fight Date"] != POSSIBLE_DATE].copy()

# ===== BEST PRICE PER FIGHTER =====
idx_best = df.groupby(["event_id","fighter"])["decimal_odds"].idxmax()
best = df.loc[idx_best, ["event_id","Matchup","Fight Date","Fight Time (UTC)","fighter","bookmaker","decimal_odds","home_team","away_team"]].copy()

# Helper: order fighters as home vs away when possible
def fighter_rows(group):
    rows = []
    ht, at = group["home_team"].iloc[0], group["away_team"].iloc[0]
    g = group.set_index("fighter")
    if ht in g.index:
        rows.append(g.loc[ht])
    if at in g.index:
        rows.append(g.loc[at])
    if len(rows) < len(group):
        seen = set([r.name for r in rows])
        for f, r in g.iterrows():
            if f not in seen:
                rows.append(r)
    return rows[:2]

# Build cards grouped by date
date_cards = {}
for (fight_date, event_id, matchup), grp in best.groupby(["Fight Date","event_id","Matchup"]):
    grp = grp.sort_values("decimal_odds", ascending=False)
    rows = fighter_rows(grp)
    if len(rows) < 1:
        continue
    time_utc = grp["Fight Time (UTC)"].iloc[0] if pd.notna(grp["Fight Time (UTC)"].iloc[0]) else ""
    f1 = rows[0].name
    b1 = rows[0]["bookmaker"]
    o1 = rows[0]["decimal_odds"]
    f2 = rows[1].name if len(rows) > 1 else ""
    b2 = rows[1]["bookmaker"] if len(rows) > 1 else ""
    o2 = rows[1]["decimal_odds"] if len(rows) > 1 else ""
    date_cards.setdefault(fight_date, []).append({
        "event_id": event_id,
        "matchup": matchup,
        "date": fight_date,
        "time_utc": time_utc,
        "f1": f1, "b1": b1, "o1": o1,
        "f2": f2, "b2": b2, "o2": o2
    })

# ===== Build upcoming_fights.html =====
def url_for_profile(fighter, matchup, date, best_price, best_book):
    params = {
        "fighter": fighter,
        "matchup": matchup,
        "date": date,
        "best_price": f"{best_price:.2f}" if pd.notna(best_price) else "",
        "best_book": best_book or ""
    }
    return "fighter_profile.html?" + urllib.parse.urlencode(params, safe="")

def card_html(c):
    def safe(x): return "" if x is None else str(x)
    fmt1 = f"{c['o1']:.2f}" if pd.notna(c["o1"]) else "-"
    fmt2 = f"{c['o2']:.2f}" if pd.notna(c["o2"]) else "-"
    href1 = url_for_profile(c["f1"], c["matchup"], c["date"], c["o1"], c["b1"])
    href2 = url_for_profile(c["f2"], c["matchup"], c["date"], c["o2"], c["b2"]) if c["f2"] else "#"
    return f"""
    <div class="fight-card">
      <div class="fight-top">
        <div class="fight-date">{c['date']}{' · ' + c['time_utc'] if c['time_utc'] else ''} UTC</div>
        <div class="fight-matchup">{safe(c['matchup'])}</div>
      </div>
      <div class="fight-odds">
        <a class="side" href="{href1}">
          <div class="name">{safe(c['f1'])}</div>
          <div class="price">Best <b>{fmt1}</b></div>
          <div class="book">{safe(c['b1'])}</div>
        </a>
        <div class="vs">vs</div>
        <a class="side" href="{href2}">
          <div class="name">{safe(c['f2'])}</div>
          <div class="price">Best <b>{fmt2}</b></div>
          <div class="book">{safe(c['b2'])}</div>
        </a>
      </div>
    </div>
    """

sections_html = ""
for fight_date in sorted(date_cards.keys()):
    cards_html = "\n".join(card_html(c) for c in date_cards[fight_date])
    sections_html += f"""
    <section>
      <h2>{fight_date}</h2>
      <div class="grid">
        {cards_html}
      </div>
    </section>
    """

upcoming_html = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8" />
<title>Upcoming Fights</title>
<meta name="viewport" content="width=device-width, initial-scale=1" />
<style>
  :root {{
    --bg: #0f1220; --card: #141a33; --muted: #9fb0d1; --line: #233157;
    --brand: #ffcc00; --accent: #00ffd0; --text: #e8ecf3;
  }}
  body {{ margin: 0; background: var(--bg); color: var(--text); font-family: Segoe UI, system-ui, -apple-system, Arial; }}
  .wrap {{ max-width: 1200px; margin: 0 auto; padding: 24px; }}
  header {{ display: flex; align-items: center; justify-content: space-between; margin-bottom: 16px; }}
  .title {{ font-size: 22px; font-weight: 800; color: var(--brand); }}
  .sub {{ color: var(--muted); font-size: 14px; }}
  h2 {{ margin: 32px 0 12px; font-size: 18px; color: var(--brand); }}
  .grid {{ display: grid; grid-template-columns: repeat(auto-fill, minmax(310px,1fr)); gap: 16px; }}
  .fight-card {{
    background: var(--card); border: 1px solid var(--line); border-radius: 14px; box-shadow: 0 12px 28px rgba(0,0,0,.25); padding: 16px; display: flex; flex-direction: column; gap: 12px;
  }}
  .fight-top {{ display: grid; gap: 6px; }}
  .fight-date {{ color: var(--muted); font-size: 13px; }}
  .fight-matchup {{ font-size: 18px; font-weight: 700; color: #cdd9ff; }}
  .fight-odds {{ display: grid; grid-template-columns: 1fr auto 1fr; align-items: center; gap: 12px; }}
  .side {{ text-decoration: none; color: inherit; padding: 10px; border-radius: 10px; border: 1px solid transparent; transition: 0.2s; }}
  .side:hover {{ background: #0f1a3a; border-color: var(--line); }}
  .side .name {{ font-weight: 700; margin-bottom: 4px; }}
  .side .price {{ color: var(--accent); margin-bottom: 2px; }}
  .side .book {{ color: var(--muted); font-size: 13px; }}
  .vs {{ font-weight: 800; color: var(--muted); }}
  footer {{ margin-top: 32px; color: var(--muted); font-size: 13px; }}
</style>
</head>
<body>
  <div class="wrap">
    <header>
      <div>
        <div class="title">Upcoming Fights</div>
        <div class="sub">Click a fighter to view their profile. Times shown in UTC.</div>
      </div>
    </header>
    {sections_html}
    <footer>Generated: {datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")} UTC · Data: odds_history (latest per book)</footer>
  </div>
</body>
</html>
"""

# Write upcoming page
os.makedirs(OUT_DIR, exist_ok=True)
with open(UPCOMING_FILE, "w", encoding="utf-8") as f:
    f.write(upcoming_html)

# ===== Build fighter_profile.html (template uses URL params) =====
fighter_profile_html = """
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8" />
<title>Fighter Profile</title>
<meta name="viewport" content="width=device-width, initial-scale=1" />
<style>
  :root {
    --bg: #0f1220; --card: #141a33; --muted: #9fb0d1; --line: #233157;
    --brand: #ffcc00; --accent: #00ffd0; --text: #e8ecf3;
  }
  body { margin:0; background:var(--bg); color:var(--text); font-family: Segoe UI, system-ui, -apple-system, Arial; }
  .wrap { max-width: 1100px; margin: 0 auto; padding: 24px; }
  header { display:flex; align-items:center; justify-content:space-between; margin-bottom: 18px; }
  .title { font-size: 22px; font-weight: 800; color: var(--brand); }
  a.back { color:#cdd9ff; text-decoration:none; background:#1f2a52; padding:10px 14px; border-radius:10px; border:1px solid var(--line); }
  .grid { display:grid; grid-template-columns: 320px 1fr; gap:18px; }
  .card { background: var(--card); border:1px solid var(--line); border-radius:14px; box-shadow:0 12px 28px rgba(0,0,0,.25); padding:16px; }
  .pic { width:100%; aspect-ratio: 3/4; background: linear-gradient(180deg, #2b335e, #192044); border-radius:12px; display:flex; align-items:center; justify-content:center; color:#9fb0d1; font-weight:700; }
  .cap { color:var(--muted); font-size:13px; margin-top:8px; }
  .name { font-size: 26px; font-weight: 800; color:#cdd9ff; }
  .meta { color: var(--muted); margin-bottom: 10px; }
  .statgrid { display:grid; grid-template-columns: repeat(3, 1fr); gap: 10px; margin-top: 12px; }
  .stat { background:#0f1a3a; border:1px solid var(--line); padding:10px; border-radius:10px; text-align:center; }
  .stat .lab { color:#9fb0d1; font-size:12px; }
  .stat .val { font-size:18px; font-weight:800; color:#e8ecf3; }
  .match-card { margin-top:16px; }
  .match-top { font-weight:700; margin-bottom:6px; color:#cdd9ff; }
  .odds { display:flex; gap:12px; align-items:center; color:#9fb0d1; }
  .odds b { color:#00ffd0; }
  .subtle { color:#9fb0d1; font-size:13px; }
  @media (max-width: 900px) {
    .grid { grid-template-columns: 1fr; }
  }
</style>
</head>
<body>
  <div class="wrap">
    <header>
      <div class="title">Fighter Profile</div>
      <a class="back" href="upcoming_fights.html">← Back to Upcoming</a>
    </header>

    <div class="grid">
      <div class="card">
        <div id="photo" class="pic">No Photo</div>
        <div id="photoCaption" class="cap"></div>
      </div>

      <div class="card">
        <div class="name" id="fighterName">Fighter Name</div>
        <div class="meta" id="metaLine">Division · Stance · Nationality · Age</div>

        <div class="statgrid">
          <div class="stat"><div class="lab">Record</div><div class="val" id="record">0-0-0</div></div>
          <div class="stat"><div class="lab">Height</div><div class="val" id="height">-</div></div>
          <div class="stat"><div class="lab">Reach</div><div class="val" id="reach">-</div></div>
        </div>

        <div class="match-card">
          <div class="match-top" id="matchup">Upcoming: -</div>
          <div class="odds">Best price: <b id="bestPrice">-</b> at <span id="bestBook">-</span></div>
          <div class="subtle" id="fightDate">Date: -</div>
        </div>
      </div>
    </div>
  </div>

<script>
  // --- parse URL params
  const p = new URLSearchParams(window.location.search);
  const fighter = p.get('fighter') || 'Unknown Fighter';
  const matchup = p.get('matchup') || '';
  const date = p.get('date') || '';
  const bestPrice = p.get('best_price') || '';
  const bestBook = p.get('best_book') || '';

  // --- placeholders (replace with real data later)
  const PLACEHOLDER_DB = {
    // Add fighter-specific overrides here later. Example:
    // "Nick Ball": { record: "20-0-0 (12 KOs)", division: "Featherweight", stance: "Orthodox", nationality: "UK", age: "26", height: "5'7\"", reach: "68\"", photo: "https://..." }
  };

  function byDefaultName(name) {
    return {
      record: "—",
      division: "—",
      stance: "—",
      nationality: "—",
      age: "—",
      height: "—",
      reach: "—",
      photo: "" // leave blank -> shows gradient block
    };
  }

  const info = PLACEHOLDER_DB[fighter] || byDefaultName(fighter);

  // --- populate UI
  document.getElementById('fighterName').textContent = fighter;
  document.getElementById('metaLine').textContent = [info.division, info.stance, info.nationality, info.age].filter(Boolean).join(" · ");
  document.getElementById('record').textContent = info.record || "—";
  document.getElementById('height').textContent = info.height || "—";
  document.getElementById('reach').textContent = info.reach || "—";
  document.getElementById('matchup').textContent = matchup ? ("Upcoming: " + matchup) : "Upcoming: —";
  document.getElementById('bestPrice').textContent = bestPrice || "—";
  document.getElementById('bestBook').textContent = bestBook || "—";
  document.getElementById('fightDate').textContent = date ? ("Date: " + date) : "Date: —";

  // Photo (placeholder gradient if none). You can map to BoxRec images later.
  if (info.photo) {
    const img = new Image();
    img.src = info.photo;
    img.alt = fighter;
    img.style.width = "100%";
    img.style.height = "100%";
    img.style.objectFit = "cover";
    img.style.borderRadius = "12px";
    const ph = document.getElementById('photo');
    ph.innerHTML = "";
    ph.appendChild(img);
    document.getElementById('photoCaption').textContent = "";
  } else {
    document.getElementById('photoCaption').textContent = "Add a photo url in PLACEHOLDER_DB to show image";
  }
</script>
</body>
</html>
"""

with open(PROFILE_FILE, "w", encoding="utf-8") as f:
    f.write(fighter_profile_html)

print(f"✅ Wrote pages:\n - {UPCOMING_FILE}\n - {PROFILE_FILE}\nOpen upcoming_fights.html and click a fighter.")


✅ Wrote pages:
 - C:\Users\chris\boxingproject\upcoming_fights.html
 - C:\Users\chris\boxingproject\fighter_profile.html
Open upcoming_fights.html and click a fighter.
